# KoELECTRA + KBMC Baseline Training

**목적**: 한국어 의료 NER baseline 구축 (GLiNER 비교용)

| 항목 | 내용 |
|------|------|
| 모델 | monologg/koelectra-base-v3-discriminator |
| 데이터 | SungJoo/KBMC (6,150 문장) |
| 라벨 | Disease, Body, Treatment (BIO) |
| 목표 | F1 ~98% (논문 수준) |

## 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 필요 패키지 설치
!pip install -q transformers datasets accelerate scikit-learn

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 설정

In [ ]:
# 모델 및 학습 설정
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
OUTPUT_DIR = "./koelectra-kbmc-baseline"

EPOCHS = 3
BATCH_SIZE = 16  # GPU면 16, CPU면 8로 조정
MAX_LENGTH = 128
LEARNING_RATE = 5e-5

# 라벨 정의 (KBMC 데이터셋 기준)
LABEL_LIST = ['O', 'Disease-B', 'Disease-I', 'Body-B', 'Body-I', 'Treatment-B', 'Treatment-I']
LABEL2ID = {label: i for i, label in enumerate(LABEL_LIST)}
ID2LABEL = {i: label for i, label in enumerate(LABEL_LIST)}

print(f"Labels: {LABEL_LIST}")
print(f"Batch size: {BATCH_SIZE}")

## 3. 데이터 로드

In [ ]:
# KBMC 데이터셋 로드
dataset = load_dataset("SungJoo/KBMC")
print(f"전체 데이터: {len(dataset['train'])} 문장")

# 샘플 확인
sample = dataset['train'][0]
print(f"\n샘플:")
print(f"  Sentence: {sample['Sentence'][:50]}...")
print(f"  Tags: {sample['Tags'][:50]}...")

In [ ]:
# Train/Test 분리 (90/10)
dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)
print(f"Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

## 4. 토크나이저 및 전처리

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer loaded: {MODEL_NAME}")

In [ ]:
def tokenize_and_align_labels(examples):
    """KBMC 데이터를 토크나이저에 맞게 정렬"""
    
    # 토큰과 태그 분리
    all_tokens = [sent.split() for sent in examples['Sentence']]
    all_tags = [tags.split() for tags in examples['Tags']]
    
    # 토크나이징
    tokenized_inputs = tokenizer(
        all_tokens,
        truncation=True,
        max_length=MAX_LENGTH,
        is_split_into_words=True,
        padding='max_length'
    )
    
    labels = []
    for i, (tokens, tags) in enumerate(zip(all_tokens, all_tags)):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        
        for word_idx in word_ids:
            if word_idx is None:
                # 특수 토큰 ([CLS], [SEP], [PAD])
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # 새로운 단어의 첫 번째 토큰
                if word_idx < len(tags):
                    label_ids.append(LABEL2ID.get(tags[word_idx], 0))
                else:
                    label_ids.append(0)  # O 태그
            else:
                # 같은 단어의 서브워드
                if word_idx < len(tags):
                    current_tag = tags[word_idx]
                    # B- 태그를 I- 태그로 변환
                    if current_tag.endswith('-B'):
                        i_tag = current_tag.replace('-B', '-I')
                        label_ids.append(LABEL2ID.get(i_tag, 0))
                    else:
                        label_ids.append(LABEL2ID.get(current_tag, 0))
                else:
                    label_ids.append(0)
            
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# 전처리 적용
print("전처리 중...")
tokenized_train = dataset['train'].map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset['train'].column_names
)

tokenized_test = dataset['test'].map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset['test'].column_names
)

print(f"전처리 완료: Train {len(tokenized_train)}, Test {len(tokenized_test)}")

## 5. 모델 로드

In [ ]:
# 모델 로드
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABEL_LIST),
    id2label=ID2LABEL,
    label2id=LABEL2ID
)

# GPU로 이동
if torch.cuda.is_available():
    model = model.cuda()
    
print(f"모델 로드 완료: {MODEL_NAME}")
print(f"파라미터 수: {model.num_parameters():,}")

## 6. 평가 함수

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    # -100 제외하고 실제 라벨만 추출
    true_labels = []
    true_predictions = []
    
    for pred, label in zip(predictions, labels):
        for p, l in zip(pred, label):
            if l != -100:
                true_labels.append(l)
                true_predictions.append(p)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, true_predictions, average='weighted', zero_division=0
    )
    accuracy = accuracy_score(true_labels, true_predictions)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## 7. 학습

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    fp16=torch.cuda.is_available(),  # GPU면 FP16 사용
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("학습 준비 완료")

In [ ]:
# 학습 실행
print("=" * 50)
print("학습 시작")
print("=" * 50)

train_result = trainer.train()

print("\n학습 완료!")

## 8. 평가

In [ ]:
# 최종 평가
eval_results = trainer.evaluate()

print("=" * 50)
print("최종 평가 결과")
print("=" * 50)
print(f"Accuracy:  {eval_results['eval_accuracy']:.4f}")
print(f"F1:        {eval_results['eval_f1']:.4f}")
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall:    {eval_results['eval_recall']:.4f}")

In [ ]:
# 모델 저장
trainer.save_model(f"{OUTPUT_DIR}/final")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final")
print(f"모델 저장: {OUTPUT_DIR}/final")

## 9. 추론 테스트

In [ ]:
def predict_ner(text, model, tokenizer):
    """단일 문장 NER 추론"""
    tokens = text.split()
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_LENGTH
    )
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.argmax(outputs.logits, dim=2)[0]
    word_ids = inputs.word_ids() if hasattr(inputs, 'word_ids') else None
    
    # 결과 매핑
    results = []
    prev_word_idx = None
    for idx, (token_id, pred) in enumerate(zip(inputs['input_ids'][0], predictions)):
        word_idx = tokenizer.convert_ids_to_tokens([token_id.item()])[0]
        if word_idx not in ['[CLS]', '[SEP]', '[PAD]']:
            label = ID2LABEL[pred.item()]
            results.append((word_idx, label))
    
    return results

# 테스트
test_text = "전신 적 다한증 은 신체 전체 에 힘 이 빠져서 일상 생활 이 어려워 지는 질환 으로"
results = predict_ner(test_text, model, tokenizer)

print("추론 테스트:")
print(f"입력: {test_text}")
print(f"\n결과:")
for token, label in results[:20]:  # 처음 20개만
    if label != 'O':
        print(f"  {token}: {label}")

## 10. 결과 요약

In [ ]:
# 결과 요약
summary = f"""
# KoELECTRA + KBMC Baseline 결과

| 항목 | 값 |
|------|----|
| 모델 | {MODEL_NAME} |
| 데이터셋 | SungJoo/KBMC |
| Train | {len(dataset['train'])} |
| Test | {len(dataset['test'])} |
| Epochs | {EPOCHS} |
| Batch Size | {BATCH_SIZE} |

## 평가 결과

| 지표 | 값 |
|------|----|
| **F1** | **{eval_results['eval_f1']:.4f}** |
| Precision | {eval_results['eval_precision']:.4f} |
| Recall | {eval_results['eval_recall']:.4f} |
| Accuracy | {eval_results['eval_accuracy']:.4f} |

## 비교 (GLiNER zero-shot)

| 모델 | F1 |
|------|----|  
| KoELECTRA (이 결과) | {eval_results['eval_f1']:.4f} |
| GLiNER zero-shot | ~0.20-0.30 (추정) |
"""

print(summary)

# 파일로 저장
with open(f"{OUTPUT_DIR}/results.md", "w", encoding="utf-8") as f:
    f.write(summary)

## 11. Google Drive 저장 (Colab용)

In [ ]:
# Colab에서 Google Drive에 저장하려면 실행
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r {OUTPUT_DIR} /content/drive/MyDrive/